In [40]:
import IPython, ipywidgets, jinja2, pandas, bokeh.plotting, traitlets
from coffeetools import coffee
from bokeh.sampledata import  iris
get_ipython = ipywidgets.get_ipython

In [9]:
%%file _layouts/d3_helper.coffee
window.update = ( selection, data, tag = 'div',  classes = [] )->
    selection = selection.selectAll [tag, classes...].join '.'
        .data data
    selection.enter().append tag
    for c in classes
        selection.classed c, true
    selection.exit().remove()
    selection

Overwriting _layouts/d3_helper.coffee


In [10]:
%%file _layouts/table.html
<table id="catalog" class="table"></table>

Overwriting _layouts/table.html


In [148]:
%%file _layouts/update_table_index.coffee
data = {{df}}
table = d3.select '#catalog'
update table, [d3.merge [[' '], data['columns']]], 'tr', ['header']
    .each (d)->
        update d3.select(@), d, 'th'
            .text (v)-> v
update table, data['data'], 'tr', ['value']
    .each (row,i)->
        update d3.select(@), [data['index'][i]], 'th'
            .text (v)-> JSON.stringify v
        update d3.select(@), row, 'td'
            .text (v)-> JSON.stringify v

Writing _layouts/update_table_index.coffee


In [149]:
%%file _layouts/update_table.coffee
data = {{df}}
table = d3.select '#catalog'
update table, [ data['columns'] ], 'tr', ['header']
    .each (d)->
        update d3.select(@), d, 'th'
            .text (v)-> v
update table, data['data'], 'tr', ['value']
    .each (row)->
        update d3.select(@), row, 'td'
            .text (v)-> JSON.stringify v

Overwriting _layouts/update_table.coffee


In [150]:
class Views( ):
    nrows = ipywidgets.IntText( description='Number of Rows', value=10 )
    row = ipywidgets.IntSlider( description='Current Row', value=0, min=0, max=1 )
    def __init__(self, df):
        self.env = jinja2.Environment( loader = jinja2.FileSystemLoader('_layouts') )
        self.df = df
        self.template = {}
        for key in self.env.list_templates():
            self.template[key] = self.env.get_template( key )
        self.columns = ipywidgets.SelectMultiple( options = df.columns.tolist(), height = 600, width = 750 )        
        self.columns.selected_labels = self.columns.options[:10]
        self.columns.observe(names='selected_labels', handler=lambda c, *args, **kwargs: self.subsample( {'new': self.row.value } ))
        self.nrows.observe(names='value', handler=lambda c, *args, **kwargs: self.subsample( {'new': self.row.value } ))
        self.row.max = self.df.shape[0]
        self.row.observe( names='value', handler= self.subsample )      
            
    def process( self, template, df = None, bare=False, display=True, *args, **kwargs):
        if isinstance(df, type(None)):
             df = self.df
        processing = self.template[template].render( df=df.to_json(orient='split'), *args, **kwargs )
        if template.endswith('.coffee'):
            processing = coffee.compile( processing, bare=bare )
        if display:
            if template.endswith('html'):
                return IPython.display.HTML( processing )
            if template.endswith('coffee') or template.endswith('js'):
                return IPython.display.Javascript( processing )
        return processing
    
    def layout( self, layout = [[]]):
        return ipywidgets.VBox(children=[ipywidgets.HBox( children=vbox ) for vbox in layout])
            
    def display( self, *args ):
        return IPython.display.display( *args )
    
    def table( self, df=None ):
        if isinstance(df, type(None)):
            self.display( self.process( self.template['table.html'].name ), 
                         self.layout([[self.nrows],[self.row]]) )
            self.display( self.subsample( {'new': self.row.value } ) )
        else:
            self.display( self.process( self.template['update_table.coffee'].name, df ) )
            
    def subsample( self, i ):
        self.table( self.df[list(self.columns.selected_labels)].iloc[i['new']:i['new']+self.nrows.value] )

In [151]:
v = Views(iris.flowers)
v.process(v.template['d3_helper.coffee'].name)

<IPython.core.display.Javascript object>

In [152]:
v.columns.height=200
v.columns

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [153]:
v.table()

<IPython.core.display.Javascript object>

None

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>